# NYC 311 Service Requests Data Pipeline

*Library + jobs to fetch, cache, and analyze NYC 311 service request data.*

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# As Jobs

## LTP Pipeline

In [2]:
from pathlib import Path

# Per-user, cross-platform base under the home dir
# retrieve for one day only:
dir_base = Path.home() / "TASK_HBC_TSY"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_poll_nyc_311 \
  --as-of=20091231 \
  --dir-base={dir_base} \
  --incremental=True \
  --log-level=INFO
%cd -q notebooks/

Log file: /Users/alexandershubert/TASK_HBC_TSY/LOGS/job_poll_nyc_311/job_poll_nyc_311_20251222150016.txt


In [3]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_poll_nyc_311 \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=5
%cd -q notebooks/

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/job_poll_nyc_311/job_poll_nyc_311_20251222150020.txt


## Analytics

In [4]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analysis_nyc_311 \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/job_analysis_nyc_311/job_analysis_nyc_311_20251222150024.txt


***

# As Library 

### Imports

In [5]:
import sys
from pathlib import Path

p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

In [6]:
import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

In [7]:
import warnings
warnings.filterwarnings("ignore")

## Api

In [8]:
from hbc import app_context, DataContainer, utils as ul

In [9]:
from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

In [10]:
app_context

AppContext
as_of : 2025-12-22
dir_analytics: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/ANALYTICS'),
dir_base: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp'),
dir_cache: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/CACHE'),
dir_logging: PosixPath('/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS')

## Logging

In [11]:
# Console-only (no file writes):
ul.conf_log(level=logging.DEBUG, console=True, file=False, reset_handlers=True)

# File-only (no console output at all):
ul.conf_log(level=logging.INFO, console=False, file=True, reset_handlers=True)

# Both:
ul.conf_log(level=logging.INFO, console=True, file=True, reset_handlers=True)

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt
Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt


<RootLogger root (INFO)>

## DataContainer

In [12]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [13]:
# we retrieve for app_context.as_of
dc.get()

2025-12-22 15:00:31 base.py                52 INFO  root    : loading...
2025-12-22 15:00:31 nycopen.py            231 INFO  root    : added filter: created_date = 2025-12-22
2025-12-22 15:00:32 nycopen.py            249 INFO  root    : Fetched 0 rows
2025-12-22 15:00:32 container.py           68 ERROR root    : DataContainer nyc_open_data_311_service_requests does not adhere to schema. Missing columns: DROP_FLAG, DROP_REASON, address_type, agency, agency_name, borough, bridge_highway_direction, bridge_highway_name, bridge_highway_segment, city, closed_date, community_board, complaint_type, created_date, cross_street_1, cross_street_2, descriptor, due_date, facility_type, ferry_direction, ferry_terminal_name, garage_lot_name, incident_address, incident_zip, intersection_street_1, intersection_street_2, landmark, latitude, location, location_type, longitude, park_borough, park_facility_name, resolution_action_updated_date, road_ramp, school_address, school_city, school_code, school_name

In [14]:
app_context.as_of = '20091231'

In [15]:
dc.get()

2025-12-22 15:00:33 base.py                52 INFO  root    : loading...
2025-12-22 15:00:33 nycopen.py            231 INFO  root    : added filter: created_date = 2009-12-31
2025-12-22 15:00:34 nycopen.py            249 INFO  root    : Fetched 4155 rows
2025-12-22 15:00:34 base.py                55 INFO  root    : cleaning...
2025-12-22 15:00:34 base.py                58 INFO  root    : normalizing...
2025-12-22 15:00:34 base.py                61 INFO  root    : validating...
2025-12-22 15:00:34 nycopen.py            188 INFO  root    : Validation summary -> flagged 311 rows. closed_date before created_date: 52; closed_date set but status not closed: 89; incident_zip outside NYC range: 4; resolution_action_updated_date before created_date: 270
2025-12-22 15:00:34 base.py                64 INFO  root    : finalizing...


In [16]:
dc.to_cache()

2025-12-22 15:00:35 cache.py               34 INFO  root    : Cached: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/CACHE/nyc_open_data_311_service_requests/20091231/nyc_open_data_311_service_requests.csv.gz


In [17]:
dc.from_cache().head()

2025-12-22 15:00:35 cache.py               53 INFO  root    : Retrieved from cache: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/CACHE/nyc_open_data_311_service_requests/20091231/nyc_open_data_311_service_requests.csv.gz


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,intersection_street_1,intersection_street_2,...,ferry_direction,ferry_terminal_name,school_or_citywide_complaint,taxi_company_borough,vehicle_type,school_not_found,garage_lot_name,landmark,DROP_FLAG,DROP_REASON
0,15654551,2009-12-31 00:00:00,2009-12-31 00:00:00,DOT,Department of Transportation,Street Condition,Pothole,10472,EAST 182 STREET,SOUTHERN BOULEVARD,...,,,,,,,,,False,
1,15631023,2009-12-31 00:00:00,2011-03-10 00:00:00,DOT,Department of Transportation,Street Light Condition,Street Light Out,,86 ST,7 ST W,...,,,,,,,,,False,
2,15632945,2009-12-31 00:00:00,2009-12-31 00:00:00,DOT,Department of Transportation,Street Light Condition,Street Light Out,,WEBSTER AVE,168 ST E,...,,,,,,,,,True,resolution_action_updated_date before created_...
3,15632946,2009-12-31 00:00:00,2009-12-31 00:00:00,DOT,Department of Transportation,Street Light Condition,Street Light Cycling,,3 AVE,167 ST E,...,,,,,,,,,True,resolution_action_updated_date before created_...
4,15619185,2009-12-31 00:00:00,2010-02-17 00:00:00,TLC,Correspondence - Taxi and Limousine Commission,Taxi Compliment,Driver Compliment,,,,...,,,,,,,,,False,


In [18]:
', '.join(dc.all_cached_dates)

'20091231'

## Analytics

In [19]:
df = dc.df

In [20]:
cols = ul.cols_as_named_tuple(df)

In [21]:
df = df[~df[cols.DROP_FLAG]]

In [22]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [23]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [24]:
path = ul.path_to_str(
                ul.mk_dir(ul.get_dir_analytics() / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

/var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/ANALYTICS/plots/closed_requests_by_location.html


In [25]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [26]:
res.keys()

dict_keys(['best', 'worst', 'median', 'mean'])

In [27]:
res['worst']

,,hbc_days_to_close
agency,agency_name,
DCA,Department of Consumer Affairs,559
TLC,Taxi and Limousine Commission,539
DOT,Department of Transportation,434
DSNY,Department of Sanitation,421
DOB,Department of Buildings,296
DEP,Department of Environmental Protection,223
HPD,Department of Housing Preservation and Development,126
DOE,Central - Department of Education,119
DOHMH,Department of Health and Mental Hygiene,61
